In [388]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore')

In [389]:
positions = pd.read_csv("portfolio_positions_exploded.csv")
costs = pd.read_csv("portfolio_costs_exploded.csv")

# filtra os custos por quem nao tem pnl nem exposiçao
costs = costs[~(costs['dtd_custos_fin'] == 0) | ~(costs['financial_value'] == 0)]

# filtra posicao pra quem nao tem pnl
positions = positions[~(positions['dtd_ativo_fin'] == 0)]

Construção do PNL de Posições

DTD

In [390]:
positions['overview_date'] = pd.to_datetime(positions['overview_date'])
costs['overview_date'] = pd.to_datetime(costs['overview_date'])
positions = positions.sort_values(['portfolio_id','instrument_id','portfolio_origem','overview_date'])

# Pego exposição e nav do dia anterior pra calcular o pnl diário
positions['exposure_value_ontem'] = (
    positions.groupby(['portfolio_id','instrument_id','portfolio_origem'])['exposure_value']
    .shift(1)
)

# DataFrame com NAV diário único por portfólio
navs = (
    positions
    .groupby(['portfolio_id', 'overview_date'])['portfolio_nav']
    .first()  # ou .max(), se sempre igual
    .reset_index()
    .sort_values(['portfolio_id', 'overview_date'])
)

# NAV de ontem
navs['portfolio_nav_ontem'] = navs.groupby('portfolio_id')['portfolio_nav'].shift(1)

# Merge no df de posições, para cada linha pegar o NAV de ontem certo para o portfolio_id e overview_date
positions = positions.merge(
    navs[['portfolio_id', 'overview_date', 'portfolio_nav_ontem']],
    on=['portfolio_id', 'overview_date'],
    how='left'
)

# Tambem faço o merge do nav_ontem em costs pra utilizar deopis
costs = costs.merge(
    navs[['portfolio_id', 'overview_date', 'portfolio_nav_ontem']],
    left_on=['root_portfolio', 'overview_date'],
    right_on=['portfolio_id', 'overview_date'],
    how='left'
).drop(columns=['portfolio_id'])

# Calculo do pnl diário do ativo e da carteira
positions['dtd_ativo_pct'] = positions['dtd_ativo_fin'] / positions['exposure_value_ontem']
positions['dtd_carteira_pct'] = positions['dtd_ativo_fin'] / positions['portfolio_nav_ontem']

MTD

In [391]:
positions['Year'] = positions['overview_date'].dt.year
positions['Month'] = positions['overview_date'].dt.month

# Calcula ano/mês do mês anterior
positions['Month_after'] = positions['Month'] + 1
positions['Year_after'] = positions['Year']
positions.loc[positions['Month_after'] == 13, 'Month_after'] = 1
positions.loc[positions['Month'] == 12, 'Year_after'] += 1

# Tabela com último exposure_value do mês para cada grupo
last_exposure_month = (
    positions
    .groupby(['portfolio_id','instrument_id','portfolio_origem','Year','Month'])
    .apply(lambda g: g.loc[g['overview_date'] == g['overview_date'].max()])
    .reset_index(drop=True)
)[['portfolio_id','instrument_id','portfolio_origem','Year_after','Month_after','exposure_value']]

last_exposure_month = last_exposure_month.rename(
    columns={'Year_after':'Year','Month_after':'Month','exposure_value':'exposure_value_prev'}
)

# Faz merge para trazer o exposure_value do mes anterior para cada linha
positions = positions.merge(
    last_exposure_month,
    on=['portfolio_id','instrument_id','portfolio_origem','Year','Month'],
    how='left'
)

# Tabela com último portfolio_nav do mês para cada grupo (faço o groupby de novo pra não incluir o instrument_id aqui)
last_nav_month = (
    positions
    .groupby(['portfolio_id','portfolio_origem','Year','Month'])
    .apply(lambda g: g.loc[g['overview_date'] == g['overview_date'].max()])
    .reset_index(drop=True)
)[['portfolio_id','portfolio_origem','Year_after','Month_after','portfolio_nav']]

last_nav_month = last_nav_month.rename(
    columns={'Year_after':'Year','Month_after':'Month', 'portfolio_nav':'portfolio_nav_prev'}
)

last_nav_month = last_nav_month.drop_duplicates()

# Faz merge para trazer o nav do mes anterior para cada linha
positions = positions.merge(
    last_nav_month,
    on=['portfolio_id','portfolio_origem','Year','Month'],
    how='left'
)

positions = positions.sort_values(['portfolio_id','instrument_id','portfolio_origem','overview_date'])


# Calcula o PnL MTD financeiro pra todos os dias:
positions['mtd_ativo_fin'] = positions.groupby(
    ['portfolio_id', 'instrument_id', 'portfolio_origem', 'Year', 'Month']
)['dtd_ativo_fin'].cumsum()

positions['mtd_ativo_pct'] = positions['mtd_ativo_fin'] / positions['exposure_value_prev']

positions['mtd_carteira_pct'] = positions['mtd_ativo_fin'] / positions['portfolio_nav_prev']

Expansão do PNL MTD para ativos que foram zerados ao longo do mês

In [ ]:
# PnL MTD
group_cols = ['portfolio_id','instrument_id','portfolio_origem','Year','Month']
all_dates = positions.groupby(group_cols)['overview_date'].agg(['min', 'max']).reset_index()

# Gera linhas para todos os dias do mês para cada ativo
expanded = []
for _, row in all_dates.iterrows():
    year = row['Year']
    month = row['Month']
    filtro = (positions['overview_date'].dt.year == year) & (positions['overview_date'].dt.month == month)

    # pega o último dia do mês de forma automática
    dates = list(positions['overview_date'][filtro].drop_duplicates())
    for date in dates:
        expanded.append({**row, 'overview_date': date})

calendar = pd.DataFrame(expanded)

# Merge calendar com positions
positions_full = pd.merge(calendar, positions, on=group_cols + ['overview_date'], how='left')

# Forward fill do PnL e das outras colunas por ativo/mês
positions_full = positions_full.sort_values(group_cols + ['overview_date'])

# positions_full['mtd_carteira_pct'] = positions_full.groupby(group_cols)['mtd_carteira_pct'].ffill()

categoria_cols = [col for col in positions_full.columns if col.startswith('grupo_')]
categoria_cols = ['mtd_ativo_fin','mtd_ativo_pct', 'mtd_carteira_pct'] + categoria_cols
positions_full[categoria_cols] = (
    positions_full.groupby(['portfolio_id', 'instrument_id', 'portfolio_origem','Year','Month'])[categoria_cols]
    .ffill()
)

YTD

In [403]:
# positions_full['Year_after'] = positions_full['Year'] + 1

# # Tabela com último exposure_value do ano para cada grupo
# last_exposure_year = (
#     positions_full
#     .groupby(['portfolio_id','instrument_id','portfolio_origem','Year'])
#     .apply(lambda g: g.loc[g['overview_date'] == g['overview_date'].max()])
#     .reset_index(drop=True)
# )[['portfolio_id','instrument_id','portfolio_origem','Year_after','exposure_value']]

# last_exposure_year = last_exposure_year.rename(
#     columns={'Year_after':'Year','exposure_value':'exposure_value_ytd_prev'}
# )

# # Faz merge para trazer o exposure_value do mes anterior para cada linha
# positions_full = positions_full.merge(
#     last_exposure_year,
#     on=['portfolio_id','instrument_id','portfolio_origem','Year'],
#     how='left'
# )

# # Tabela com último portfolio_nav do mês para cada grupo (faço o groupby de novo pra não incluir o instrument_id aqui)
# last_nav_year = (
#     positions_full
#     .groupby(['portfolio_id','portfolio_origem','Year'])
#     .apply(lambda g: g.loc[g['overview_date'] == g['overview_date'].max()])
#     .reset_index(drop=True)
# )[['portfolio_id','portfolio_origem','Year_after','portfolio_nav']]

# last_nav_year = last_nav_year.rename(
#     columns={'Year_after':'Year','portfolio_nav':'portfolio_nav_ytd_prev'}
# )

# last_nav_year = last_nav_year.drop_duplicates()
# last_nav_year = last_nav_year.dropna()

# # Faz merge para trazer o nav do mes anterior para cada linha
# positions_full = positions_full.merge(
#     last_nav_year,
#     on=['portfolio_id','portfolio_origem','Year'],
#     how='left'
# )

# positions_full = positions_full.sort_values(['portfolio_id','instrument_id','portfolio_origem','overview_date'])

# Calcula o PnL ytd financeiro pra todos os dias:
positions_full['ytd_ativo_fin'] = positions_full.groupby(
    ['portfolio_id', 'instrument_id', 'portfolio_origem', 'Year']
)['dtd_ativo_fin'].cumsum()

# positions_full['ytd_ativo_pct'] = positions_full['ytd_ativo_fin'] / positions_full['exposure_value_ytd_prev']

# positions_full['ytd_carteira_pct'] = positions_full['ytd_ativo_fin'] / positions_full['portfolio_nav_ytd_prev']

positions_full.drop(columns=(['min','max']),inplace=True)

Tratamento do DF para exportação


In [404]:
positions_full_filtrado = positions_full[[
    'portfolio_id','portfolio_origem','book_name','overview_date','instrument_id',
    'dtd_ativo_fin','dtd_ativo_pct','dtd_carteira_pct','ytd_ativo_fin'] + categoria_cols]
positions_full_filtrado.replace([np.inf, -np.inf], 0, inplace=True)

Construção do PNL de Custos

In [405]:
costs['overview_date'] = pd.to_datetime(costs['overview_date'])

costs['Year'] = costs['overview_date'].dt.year
costs['Month'] = costs['overview_date'].dt.month

costs = costs.sort_values(['origin_portfolio_id', 'root_portfolio', 'overview_date'])

# Calcula ano/mês do mês anterior
costs['Month_after'] = costs['Month'] + 1
costs['Year_after'] = costs['Year']
costs.loc[costs['Month_after'] == 13, 'Month_after'] = 1
costs.loc[costs['Month'] == 12, 'Year_after'] += 1

# Faz merge para trazer o nav do mes anterior para cada linha
costs = costs.merge(
    last_nav_month,
    left_on=['root_portfolio','origin_portfolio_id','Year','Month'],
    right_on=['portfolio_id','portfolio_origem','Year','Month'],
    how='left'
)

costs = costs.sort_values(['category_name','origin_portfolio_id','root_portfolio','overview_date'])


# Calcula o PnL MTD pra qualquer dia:
costs['mtd_custos_fin'] = costs.groupby(
    ['category_name','origin_portfolio_id','root_portfolio','Year','Month']
)['dtd_custos_fin'].cumsum()

costs['ytd_custos_fin'] = costs.groupby(
    ['category_name','origin_portfolio_id','root_portfolio','Year']
)['dtd_custos_fin'].cumsum()

costs['dtd_custos_pct'] = costs['dtd_custos_fin'] / costs['portfolio_nav_ontem']
costs['mtd_custos_pct'] = costs['mtd_custos_fin'] / costs['portfolio_nav_prev']

In [406]:
# PnL MTD
group_cols = ['category_name','origin_portfolio_id','root_portfolio','Year','Month']
all_dates = costs.groupby(group_cols)['overview_date'].agg(['min', 'max']).reset_index()

# Gera linhas para todos os dias do mês para cada ativo
expanded = []
for _, row in all_dates.iterrows():
    year = row['Year']
    month = row['Month']
    filtro = (costs['overview_date'].dt.year == year) & (costs['overview_date'].dt.month == month)

    # pega o último dia do mês de forma automática
    dates = list(costs['overview_date'][filtro].drop_duplicates())
    for date in dates:
        expanded.append({**row, 'overview_date': date})

calendar = pd.DataFrame(expanded)

# Passo 2: Merge calendar com costs
costs_full = pd.merge(calendar, costs, on=group_cols + ['overview_date'], how='left')

# Passo 3: Forward fill do PnL e das outras colunas por ativo/mês
costs_full = costs_full.sort_values(group_cols + ['overview_date'])
costs_full['mtd_custos_pct'] = costs_full.groupby(group_cols)['mtd_custos_pct'].ffill()

costs_full['book_name'] = "Caixas e Provisionamentos"

# Tratamento do DF para exportação
costs_full_filtrado = costs_full[[
    'overview_date','category_name','book_name','origin_portfolio_id','root_portfolio',
    'dtd_custos_fin', 'portfolio_id',
    'mtd_custos_fin','dtd_custos_pct','mtd_custos_pct','ytd_custos_fin']]

costs_full_filtrado.replace([np.inf, -np.inf], 0, inplace=True)

Concatena a tabela no csv de posições e custos, reprocessando as datas coincidentes

In [408]:
try:
    main_csv_position = pd.read_csv("positions_pnl_history.csv",parse_dates=['overview_date'])
    main_csv_costs = pd.read_csv("costs_pnl_history.csv",parse_dates=['overview_date'])

    main_csv_position.to_csv("positions_pnl_history_backup.csv", index=False)
    main_csv_costs.to_csv("costs_pnl_history_backup.csv", index=False)

    """ Para posições: """
    # Primeiro: remove do main_csv_position todas as linhas que têm datas que existem no positions_full_filtrado
    main_csv_filtrado = main_csv_position[~main_csv_position['overview_date'].isin(positions_full_filtrado['overview_date'])]

    # Segundo: concatena os dois
    df_resultado = pd.concat([main_csv_filtrado, positions_full_filtrado], ignore_index=True)

    # Terceiro: opcional - reordenar por data, se quiser
    df_resultado = df_resultado.sort_values('overview_date').reset_index(drop=True)

    #Exporta CSV
    df_resultado.to_csv("positions_pnl_history.csv", index=False)


    """ Para custos: """
    # Primeiro: remove do main_csv_costs todas as linhas que têm datas que existem no costs_full_filtrado
    main_csv_filtrado = main_csv_costs[~main_csv_costs['overview_date'].isin(costs_full_filtrado['overview_date'])]

    # Segundo: concatena os dois
    df_resultado = pd.concat([main_csv_filtrado, costs_full_filtrado], ignore_index=True)

    # Terceiro: opcional - reordenar por data, se quiser
    df_resultado = df_resultado.sort_values('overview_date').reset_index(drop=True)

    #Exporta CSV
    df_resultado.to_csv("costs_pnl_history.csv", index=False)

except Exception as e:
    print(f"Error: {e}")
    print("Not processed.")
    #Exporta CSV
    # positions_full_filtrado.to_csv("positions_pnl_history.csv", index=False)
    # costs_full_filtrado.to_csv("costs_pnl_history.csv", index=False)